In [42]:
!clingo q0.db datalog_query0.lp

clingo version 5.6.2
Reading from q0.db ...
Solving...
Answer: 1
satisfied
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 3.406s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 3.313s


In [47]:
!clingo query0.db gac_query0.lp

clingo version 5.6.2
Reading from query0.db ...
Solving...
UNSATISFIABLE

Models       : 0
Calls        : 1
Time         : 0.002s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


In [53]:
!clingo query1.db datalog_query1.lp

clingo version 5.6.2
Reading from query1.db ...
Solving...
Answer: 1
satisfied
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.007s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


In [54]:
!clingo query1.db gac_query1.lp

clingo version 5.6.2
Reading from query1.db ...
Solving...
Answer: 1
p(1,1,1,1) p(2,2,3,2) vrow(100,r,x,x) vrow(100,r,y,y) vrow(100,r,z,z) vrow(100,r,w,w) kd(1,r,x) kd(2,r,y) kd(3,r,z)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.001s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


In [40]:
f = open("q0.db", "w")

i=0
result_datalog=[]
result_gac=[]
while(i<100000):
    line=f'p({i},{2*(i)},{3*i}). s({2*i},0). r({3*i},{i+3},c). '
    f.write(line)
    
    if(i%10000==0):
        result = subprocess.run(['clingo','q0.db',"datalog_query0.lp"], stdout=subprocess.PIPE, universal_newlines=True)
        result_datalog.append(result.stdout.split("CPU Time     : ")[1].split("s")[0])

        result = subprocess.run(['clingo','q0.db',"gac_query0.lp"], stdout=subprocess.PIPE, universal_newlines=True)
        result_datalog.append(result.stdout.split("CPU Time     : ")[1].split("s")[0])
    i=i+1


f.close()

In [41]:
result_datalog

['0.000',
 '0.000',
 '0.359',
 '1.250',
 '0.594',
 '2.313',
 '1.109',
 '3.922',
 '1.203',
 '5.438',
 '1.938',
 '7.188',
 '2.281',
 '8.000',
 '2.109',
 '6.156',
 '1.500',
 '8.453',
 '3.188',
 '7.422']